# Metaprogramming

Julia nos permite manipular objetos en Julia que representan código en Julia. De esta manesa, podemos producir código de una manera automática.

Consideremos los polinomios de Wilkinson

$$ p_{20} (x) := (x-1)\cdot(x-2)\dotsm (x-20) = \prod_{i=1}^{20} (x-i) $$ 

Supongamos que queremos definir este polinomio en Julia. La forma más simple sería explícitamente

In [ ]:
p_5(x) = (x-1)*(x-2)*(x-3)*(x-4)*(x-5)

Pero se sale de control muy rápidamente esta forma de hacer las cosas, para el momento en que lleguemos a $p_{30}$ estaremos deseando que exista una forma más fácil de hacerlo. El punto es que, la hay

In [ ]:
function wilkinson(n, x)
    result = x - 1
    
    for i in 2:n
        result *= (x - i)
    end
    
    result
end

Podemos usar una función anónima, para tener la función objeto $p_n$:

In [ ]:
wilkinson(n) = x -> wilkinson(n, x)

Sin embargo las funciones anónimas son lentas. Nosotros desearíamos que hubiera alguna forma de hacer un loop que escriba el código de nuestra función

## Expresiones

En otros lenguajes de programación, se manipula al código como cadenas. Julia aborda el problema desde otro punto. Consideremos la cadena

In [ ]:
s = "(x-1) * (x-2)"

Para convertir esto a un objeto usamos **parse**:

In [ ]:
ex = parse(s)

In [ ]:
typeof(ans)

*ex* es un objeto de tipo Expr. Puede pensarse como la representación de un "árbol de sintaxis abstracta" representando la estructura interna de la expresión. Podemos ver esto de dos maneras, utilizando `dump`

In [ ]:
dump(ex)

que muestra todo en detalle, o

In [ ]:
Meta.show_sexpr(ex)

que nos muestra una versión compacta.

vemos además que los objetos `Expr` tienen un formato jerárquico que representa el código. Dado que son simplemente objetos de Julia, podemos utilizar Julia para manipularlos.

## Interpolación de código

En nuestro caso no necesitamos meternos con la estructura interna del código, sino construir el código a partir de bits preexistentes

In [ ]:
ex = :(x-1)

In [ ]:
ex = :(($ex)*(x - 2))

Aquí, del mismo modo que en la interpolación de cadenas, hemos insertado el valor actual de ex dentro de la expresión

In [ ]:
ex = :(($ex)*(x - 3))

Ahora podemos hacer nuestro loop

In [ ]:
n = 10
ex = :(x-1)
for i in 2:n
    ex = :( ($ex) * (x-i) )
end

In [ ]:
ex

Pero lo que nosotros queremos es el valor, no el código

In [ ]:
n = 10
ex = :(x-1)
for i in 2:n
    ex = :( ($ex) * (x-$i) )
end

In [ ]:
ex

Lo que necesitamos ahora es dar nombre a la función

In [ ]:
nombre = symbol(string("W_", n))

Así que el código se ve más o menos así

In [ ]:
código = :( $nombre(x) = $ex )

Ahora deseamos evaluar eso:

In [ ]:
eval(código)

Comparemos las dos funciones evaluándolas en una malla dada

In [ ]:
f1(range) = [W_10(x) for x in range]
f2(range) = [wilkinson(10, x) for x in range]

In [ ]:
range = -10:0.000001:10
@time f1(range);
@time f2(range);

## Macros

Hasta ahora hemos estado usando ciertos objetos con un @ que los precede, uno estaría tentado a decir que son funciones; sin embargo en el sentido estricto de la palabra no son funciones sino **macros**. Que son funciones cuyos argumentos son código, y que transforman una expresión de código en otra. Esta nueva expresión es entonces evaluada como si el nuevo código hubiera sido tecleado directamente.

Para ver con más claridad esto definamos una macro

In [ ]:
macro simple(expr)
    @show expr
    0  # for the moment
end

In [ ]:
@simple x+y

vemos que el código de Julia que sigue a la llamada de la macro es pasado a la macro como si ya le hubiéramos hecho *parse*.

Supongan que ahora redefinimos la macro como

In [ ]:
macro simple(expr)
    @show expr
    expr  # returns expr
end

Entonces tenemos

In [ ]:
@simple x+y

Lo que pasa es que la macro regresa :(x+y), y esto es evaluado utilizando `eval`. El resultado es que Julia intenta calcular el valor de la expresión x+y, pero la variable x no está definida. Definamos $y$ y $z$, pero no $x$:

In [ ]:
y = 3; z = 4

In [ ]:
x

In [ ]:
@simple x+y

In [ ]:
?esc

## Un último ejemplo

Supongamos que queremos evaluar el polinomio 

$$ p(x) = a_n x^n + a_{n-1}x^{n-1} + \dotsm + a_1x^1 + a_0  $$

una forma eficiente de hacer esto es con el [método de Honer](https://en.wikipedia.org/wiki/Horner%27s_method), de la siguiente forma

$$ p(x) = a_0 + x(a_1 + x(a_{n-2} + \dotsm + x(a_{n-1} + xa_n))) $$

In [ ]:
# copiado de base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

Para evaluar el polinomio $p(x) = 2+3x+4x^2$ en $x=3$, hacemos

In [ ]:
x = 3
@horner(x, 2, 3, 4)

# Graficando, salvando y leyendo de un archivo

Pasando a otras cosas, creemos un arreglo de números aleatorios

In [ ]:
using PyPlot

In [ ]:
x = [1:1:100]
arreglo1 = rand(100)
arreglo2 = rand(100)

In [ ]:
plot(x,arreglo1, "rp", label = "Arreglo 1")
plot(x,arreglo2, "g*", label = "Arreglo 2")
legend(loc="upper right",fancybox="true")
axis([0,100,0,1])
title("Gráfica sencilla")
ylabel("Número generado", size = 16)
xlabel(L"x", size = 18)
grid("on")

In [ ]:
fig = figure("pyplot_subplot_mixed",figsize=(14,7))
subplot(131)
grid("on")
bar(x,arreglo1)
axis([0,100,0,1])
title("Barras")
subplot(132)
title("Barras horizontales")
barh(x,arreglo2)
axis([0,1,0,100])
ax = subplot(133)
plot(x,arreglo1)
plot(x,arreglo2)
title("Dos a la vez");

Para grabar estos arreglos en un archivo hacemos

In [ ]:
writedlm("arreglo1.txt", arreglo1)
writedlm("arreglo2.txt", arreglo2)

Para leer utilizamos

In [ ]:
arregloLeido1 = readdlm("arreglo1_prueba.txt")
arregloLeido2 = readdlm("arreglo2_prueba.txt");

Para que vean que no hay trampa

In [ ]:
plot(x,arregloLeido1, "g^", label = "Arreglo 1")
plot(x,arregloLeido2, "bh", label = "Arreglo 2")
legend(loc="upper right",fancybox="true")
axis([0,100,0,1])
title("Arreglos leídos")
ylabel("Número generado", size = 16)
xlabel(L"x", size = 18)
grid("on")

# Referencias

1. [Introducing Julia](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming)
2. [Documentación de Julia](http://julia.readthedocs.org/en/latest/manual/metaprogramming/)}
3. [D. Sanders - Invitation to julia](https://github.com/dpsanders/invitation_to_julia/blob/master/3.%20Metaprogramming.ipynb)
4. [D. Sanders - ScyPy 2014](https://github.com/dpsanders/scipy_2014_julia/blob/master/Metaprogramming.ipynb)